# COSI123A Final Project
### Group Name: NKG
### Group Members: Nicole Meng, Kevin Dai, Grant Gu

In [1]:
import numpy as np
import pandas as pd
import csv
import math
from sklearn.model_selection import train_test_split

## Import and Split Data

In [2]:
test = pd.read_csv("test.csv", header = None)
train = pd.read_csv("train.csv", header = None)
train_updates = pd.read_csv("train_updates_20220929.csv", header = None)

#converting the types
test[0][1:] = test[0][1:].astype(float).astype(int)
train[0][1:] = train[0][1:].astype(float).astype(int)
train[2][1:] = train[2][1:].astype(float)
train[4][1:] = train[4][1:].astype(float)
train_updates[0][1:]= train_updates[0][1:].astype(float).astype(int)
train_updates[2][1:] = train_updates[2][1:].astype(float)
train_updates[4][1:] = train_updates[4][1:].astype(float)

In [3]:
#update train to contain the correct training data
for i in range(1, 2435):
    fix_id = train_updates[0][i] #the id that needs to be fixed
    train.iloc[fix_id + 1] = train_updates.iloc[i]

In [4]:
#drop irrelevant column - data source
train = train.drop(columns = [3])
#drop invalid pH values
for i in range(1, 31391):
    ph = train[2][i] #the ph value
    if ph < 0 or ph > 14:
        train.drop(i)

In [5]:
#We are planning on performing cross validation on the dataset with a 80-20 train-test split
X_train=train.sample(frac=0.8,random_state=200)
X_test=train.drop(X_train.index)

In [6]:
#Next, we will read into the dataframes we created and define some more terms
X_train_x = X_train.iloc[: , 0:-1] #80% trainng data used for training model
X_train_y = X_train.iloc[: , -1] #80% target column

X_test_x = X_test.iloc[: , 0:-1] #20% training data used for training model
X_test_y = X_test.iloc[: , -1] #20% training data used for training model

#Note: for our model training, we will only use X_train_x and X_train_y

# Data Preprocessing
Drop the columns that is wrong or does not provide useful inforamtion

Drop rows that has Nan in sequence column. 
Add a wild type column 
Add a group column to store the group this sample is assigned to

In [7]:
#drop nan rows
X_train_x=X_train_x[X_train_x[1].notna()]
print(X_train_x)
wild_col=[' ']*23166 #add a wild type column
X_train_x.insert(loc=1,column='wild',value=wild_col)
group_col=[0]*23166
X_train_x.insert(loc=1,column='group',value=group_col)
X_train_x.insert(loc=1,column='seq_len',value=X_train_x[1].str.len())
#drop sequences of uniq length
print(X_train_x)
X_train_x = X_train_x.loc[X_train_x.duplicated(subset='seq_len', keep=False), :]
print(X_train_x)
#drop sequences of length appear less than 20 times
indexes=X_train_x.seq_len.value_counts().loc[lambda x: x>20].reset_index()['index']
#print(indexes)
for index in indexes:
    X_train_x.drop(X_train_x[X_train_x['seq_len']==index].index, inplace=True)
print(X_train_x)


           0                                                  1    2
13355  13354  MKALIVLGLVLLSVTVQGKVFERCELARTLKRLGMDGYRGISLANW...  2.7
2917    2916  MAAALPLQPSTTASATTTATAVALGEVEDEGLLASLFRDRFPEAQW...  7.0
6453    6452  MASSPKCLGKECSAPGPVPNGQIRLYSMRFCPFAQRTRLVLTAKGV...  7.0
4164    4163  MAELGLNEHHQNEVINYMRFARSKRGLRLKTVDSCFQDLKDSRLVE...  7.0
31273  31272  YFIMACDQYSCSLTAPALDIATGHASLADIWAKTPPVTAKAAQLYA...  7.0
...      ...                                                ...  ...
24692  24691  MSLLATPMPQAATASSSSSASAAASASGIPITANNNLPFEKDKIWY...  7.0
10614  10613  MFLTRSEYDRGVSTFSPEGRLFQVEYSLEAIKLGSTAIGIATKEGV...  7.0
24292  24291  MSKIQVAHSSRLANLIDYKLRVLTQDGRVYIGQLMAFDKHMNLVLN...  7.0
9356    9355  MELGRSPSTPQEEAISPLGVLGTGPSSSPLGKLQALPIGPGAHRGA...  7.0
14839  14838  MKQEKERILKLVEEGKLTAQEALTLIEKLDSDYKEKEEKITALSVH...  7.0

[23166 rows x 3 columns]
           0  seq_len  group wild  \
13355  13354      148      0        
2917    2916      639      0        
6453    6452      239      0       

create sequence and wild type columns 

In [8]:
seq_col=X_train_x.loc[:,1]
print(seq_col)
wild_col=X_train_x.loc[:,'wild']
print(wild_col)

2917     MAAALPLQPSTTASATTTATAVALGEVEDEGLLASLFRDRFPEAQW...
26802    MTDLFASGDNSSNSFDDSNDEIARKNETAFKYTTIFHTNLSEKLYH...
29098    MVRVKVNDRIVEVPPGTSVMDAVFHAGYDVPLFCSEKHLSPIGACR...
8866     MEESSVTVGTIDVSYLPSSSEYSLGRCKHTSEDWVDCGFKPTFFRS...
8392     MDVSGQETDWRSAAFRQKLVSQIEDAMRKAGVAHSKSSKDMESHVF...
                               ...                        
14786    MKPNIHPEYRTVVFHDTSVDEYFKIGSTIKTDREIELDGVTYPYVT...
2740     LSLWCPSSPSLARRFPARFSPIGSRIASRSLVTASFANENREFVIV...
9337     MELDPDDVFRDEDEDPENDFFQEKEASKEFVVYLIDASPKMFCSTC...
24692    MSLLATPMPQAATASSSSSASAAASASGIPITANNNLPFEKDKIWY...
9356     MELGRSPSTPQEEAISPLGVLGTGPSSSPLGKLQALPIGPGAHRGA...
Name: 1, Length: 6041, dtype: object
2917      
26802     
29098     
8866      
8392      
        ..
14786     
2740      
9337      
24692     
9356      
Name: wild, Length: 6041, dtype: object


Compare sequences to see if they differ by less than 2 mutation

In [9]:
def compare_seq(seq1,seq2):
    if(type(seq1)!=str):
        return 0
    if(type(seq2)!=str):
        return 0
    if(len(seq1)>len(seq2)+3 or len(seq1)<len(seq2)-3):
        return 0
    #print('seq1 len',len(seq1))
    count=0
    minlen=min(len(seq1),len(seq2))
    for i in range(minlen):
        if seq1[i]!=seq2[i]:
            count=count+1
    if count>3:
        return 0
    #print(count)
    return 1

In [10]:
def is_subseq(seq1, seq2):
    it = iter(seq2)
    return all(any(c1 == c2 for c1 in it) for c2 in seq1)

sort dataframe base on sequence, set wildtype to temp cell and adjust later

In [11]:
group_num=1

#print(X_train_x['group'].loc[3])
for i in wild_col.index:
    current_wild=seq_col[i]
    if wild_col[i]==' ':
        print('find group for ',group_num)
        for j in wild_col.index:
            if (X_train_x.at[j,'group']==0) and (compare_seq(current_wild,seq_col[j])==1 or is_subseq(current_wild,seq_col[j])):#found relative
                X_train_x.at[j,'wild']=current_wild
                X_train_x.at[j,'group']=group_num
        group_num=group_num+1
print(X_train_x['group'].head(1000))

find group for  1
find group for  2
find group for  3
find group for  4
find group for  5
find group for  6
find group for  7
find group for  8
find group for  9
find group for  10
find group for  11
find group for  12
find group for  13
find group for  14
find group for  15
find group for  16
find group for  17
find group for  18
find group for  19
find group for  20
find group for  21
find group for  22
find group for  23
find group for  24
find group for  25
find group for  26
find group for  27
find group for  28
find group for  29
find group for  30
find group for  31
find group for  32
find group for  33
find group for  34
find group for  35
find group for  36
find group for  37
find group for  38
find group for  39
find group for  40
find group for  41
find group for  42
find group for  43
find group for  44
find group for  45
find group for  46
find group for  47
find group for  48
find group for  49
find group for  50
find group for  51
find group for  52
find group for  53
fi

find group for  417
find group for  418
find group for  419
find group for  420
find group for  421
find group for  422
find group for  423
find group for  424
find group for  425
find group for  426
find group for  427
find group for  428
find group for  429
find group for  430
find group for  431
find group for  432
find group for  433
find group for  434
find group for  435
find group for  436
find group for  437
find group for  438
find group for  439
find group for  440
find group for  441
find group for  442
find group for  443
find group for  444
find group for  445
find group for  446
find group for  447
find group for  448
find group for  449
find group for  450
find group for  451
find group for  452
find group for  453
find group for  454
find group for  455
find group for  456
find group for  457
find group for  458
find group for  459
find group for  460
find group for  461
find group for  462
find group for  463
find group for  464
find group for  465
find group for  466


find group for  827
find group for  828
find group for  829
find group for  830
find group for  831
find group for  832
find group for  833
find group for  834
find group for  835
find group for  836
find group for  837
find group for  838
find group for  839
find group for  840
find group for  841
find group for  842
find group for  843
find group for  844
find group for  845
find group for  846
find group for  847
find group for  848
find group for  849
find group for  850
find group for  851
find group for  852
find group for  853
find group for  854
find group for  855
find group for  856
find group for  857
find group for  858
find group for  859
find group for  860
find group for  861
find group for  862
find group for  863
find group for  864
find group for  865
find group for  866
find group for  867
find group for  868
find group for  869
find group for  870
find group for  871
find group for  872
find group for  873
find group for  874
find group for  875
find group for  876


find group for  1226
find group for  1227
find group for  1228
find group for  1229
find group for  1230
find group for  1231
find group for  1232
find group for  1233
find group for  1234
find group for  1235
find group for  1236
find group for  1237
find group for  1238
find group for  1239
find group for  1240
find group for  1241
find group for  1242
find group for  1243
find group for  1244
find group for  1245
find group for  1246
find group for  1247
find group for  1248
find group for  1249
find group for  1250
find group for  1251
find group for  1252
find group for  1253
find group for  1254
find group for  1255
find group for  1256
find group for  1257
find group for  1258
find group for  1259
find group for  1260
find group for  1261
find group for  1262
find group for  1263
find group for  1264
find group for  1265
find group for  1266
find group for  1267
find group for  1268
find group for  1269
find group for  1270
find group for  1271
find group for  1272
find group fo

KeyError: 1

MAAALPLQPSTTASATTTATAVALGEVEDEGLLASLFRDRFPEAQWREKPDVGRYLRELSGSGLDRLRREPERLAEERAQRLQQTRDLAFANYKTFIRGAECTERIHRLFGDVEASLGRLLDRLPRFQQSCRNFVKEAEEISSSRRMNTLTLNRHTEILEILEIPQLMDTCVRNSYHEEALELAAYVRRLERKYSSIPVIQGIVNEVRQSMQLMLSQLIQQLRTNIQLPACLRVIGYLRRMDVFTEAELRVKFLQARDAWLRSILTAIPNDDPYFHITKTIEACRVHLFDIITQYRAIFSDEDPLLPPAMGEYTVNEGAIFHGWVLQKISQFLQVLETDLYRGIGGRLDSLLGQCMYFGLSFSRVGADFRGQLAPVFQRVAISTFQKAVEEAVEKFQDEMTSYTLISTAAILGSSNTPATVPATQPGTLQPPMVLLDFPPLACFLNNILVAFNDLRLCCPVALAQDVTGTLENALTKVTKTILAFHRAEEAVFSSGEHEIFVQFCTAFLEDLVPYLNRCLQVLFPPAQIAQTLGISPTQLSKHGNLGHVNISAIQEPLAFILPKRETVFCLDEQELGPDLVAPAPELPAEQRSMEPVTEKREPGEPLPQEPMEGEPLPAEPPSEGGAIGSVPCPQPGEQ


function used groupby to group the similar sequences(less than 2 mutations apart)